In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.integrate import quad
from scipy.optimize import minimize
import datetime
from dateutil.tz import tzutc


# -------------------------
# Non-vectorized calculation of Heston call price (to be called in 'Heston_calibration' as the regression function.
# -------------------------
def heston_call(S0, K, v0, r, t, kappa, theta, xi, rho, phi_max=200):
    """
    Heston call price using Carr-Madan style integrals (characteristic function).
    phi_max: upper limit of integral (increase for more accuracy).
    """
    i = 1j

    def integrand(phi, Pnum): 
        u = 0.5 if Pnum == 1 else -0.5
        b = kappa - rho * xi if Pnum == 1 else kappa
        a = kappa * theta
        # complex sqrt
        d = np.sqrt((rho * xi * phi * i - b) ** 2 - (xi ** 2) * (2 * u * phi * i - phi ** 2))
        g = (b - rho * xi * phi * i + d) / (b - rho * xi * phi * i - d)

        exp1 = np.exp(i * phi * np.log(S0 / K))
        # C and D as in Heston closed form
        C = r * phi * i * t + (a / (xi ** 2)) * ((b - rho * xi * phi * i + d) * t - 2 * np.log((1 - g * np.exp(d * t)) / (1 - g)))
        D = ((b - rho * xi * phi * i + d) / (xi ** 2)) * ((1 - np.exp(d * t)) / (1 - g * np.exp(d * t)))
        f = exp1 * np.exp(C + D * v0)
        # real part of f / (i * phi)
        return np.real(f / (i * phi))

    # numeric integrals from epsilon to phi_max
    eps = 1e-8
    P1 = 0.5 + (1 / np.pi) * quad(lambda ph: integrand(ph, 1), eps, phi_max, limit=200)[0]
    P2 = 0.5 + (1 / np.pi) * quad(lambda ph: integrand(ph, 2), eps, phi_max, limit=200)[0]
    call_price = S0 * P1 - K * np.exp(-r * t) * P2             # This is the semi-closed form of European Heston call price.
    return call_price



def find_tte_yf_options(expiration_date, last_trade_date):      
    """
    Return time to expiration in years between an expiration date string 'YYYY-MM-DD'
    and yfinance lastTradeDate (pd.Timestamp, possibly tz-aware).
    We assume expiration time at 21:30 UTC (close-of-day-ish); adjust if you used different convention.
    """
    # construct tz-aware expiration datetime at 21:30 UTC
    exp_dt = datetime.datetime.strptime(expiration_date, "%Y-%m-%d").replace(hour=21, minute=30, tzinfo=tzutc())
    # ensure last_trade_date is timezone-aware (if naive assume UTC)
    if last_trade_date.tzinfo is None:
        lt = last_trade_date.replace(tzinfo=tzutc())
    else:
        lt = last_trade_date
    seconds = (exp_dt - lt).total_seconds()
    years = seconds / (60 * 60 * 24 * 365)
    return years


def yf_find_approx_spot(stock_data, last_trade_date):
    """
    Approximate spot price at last_trade_date using minute-level yfinance data.
    stock_data: Series or DataFrame with DatetimeIndex (minute resolution). Prefer column 'Close' if DataFrame.
    last_trade_date: pd.Timestamp
    """
    # round to minute
    ts = last_trade_date.replace(second=0, microsecond=0)
    # try exact minute
    try:
        if isinstance(stock_data, pd.DataFrame):
            if 'Close' in stock_data.columns:
                return float(stock_data.loc[ts, 'Close'])
            # otherwise take first column
            return float(stock_data.loc[ts].iloc[0])
        else:
            # Series
            return float(stock_data.loc[ts])
    except Exception:
        # If exact minute missing, try nearest minute within +/- 1 minute
        for delta_sec in (60, -60, 120, -120):
            try_ts = ts + pd.Timedelta(seconds=delta_sec)
            try:
                if isinstance(stock_data, pd.DataFrame):
                    if 'Close' in stock_data.columns:
                        return float(stock_data.loc[try_ts, 'Close'])
                    return float(stock_data.loc[try_ts].iloc[0])
                else:
                    return float(stock_data.loc[try_ts])
            except Exception:
                continue
    return float('nan')


# -------------------------
# Calibration wrapper (keeps same structure) - minimize squared price errors
# -------------------------
def Heston_calibration(ticker_str, r):
    """
    This is the function that captures the Heston paramters for a given stock under discount rate r, that is
    returning the parameters [kappa, theta, xi, rho, v0] assuming the stock movement is well fitted into Heston's model.
    """
    ticker_str = 'NFLX'
    ticker = yf.Ticker(ticker_str)
    stock_data = yf.download(ticker_str, period = '1d', interval = '1m')

    r = 0.039          # This is the discount rate.

    expirations = ticker.options  # This collects the option expiration dates for ticker.

    # We first clean the call option data.
    option_data = []     # Creating an empty data frame to store stock data.

    for date in expirations:
        chain = ticker.option_chain(date)   # Get the stock chain on date'.
    
        # Add expiration and label
        calls = chain.calls.copy()   # Creating a calls data frame for call options, and add expiration date as well as option type.
        calls['expiration'] = date
        calls['option_type'] = 'call'
    
        option_data.append(calls) 

    options_data = pd.concat(option_data, ignore_index=True)

    options_data = options_data.drop(columns = ['impliedVolatility']) # Delete the implied volatility column, as we will recalculate that (the one given by yfinance is not reliable).
    
    # Delete any options that were traded in the interval of historical stock values obtained
    start_date = stock_data.index[0]
    options_data = options_data[options_data['lastTradeDate']>=start_date]
    
    
    #Insert column of time to expiration in years of the option contract measured from time of last trade
    options_data['time_to_expiration'] = options_data.apply(
    lambda row: find_tte_yf_options(expiration_date = row['expiration'],              # Calling the function that calculates the time till expiration.
                                   last_trade_date = row['lastTradeDate']),
        axis = 1
    )
    
    
    #Add in column of the spot price of stock when the option trade occured.
    options_data['spot_price'] = options_data.apply(
        lambda row: yf_find_approx_spot(stock_data['Close'], row['lastTradeDate']),
        axis=1
    )
    options_data = options_data.dropna()
    
    #Create data frames that keeps relevant information and separate calls from puts.
    options_data = options_data[['strike', 'lastPrice', 'lastTradeDate',\
                                 'expiration', 'option_type','time_to_expiration', 'spot_price']]
    options_data_calls = options_data[(options_data['option_type'] == 'call')].copy()
  
    
    
    #Remove rows with undefined values
    options_data_calls = options_data_calls.dropna()

    options_data_calls = options_data_calls[(options_data_calls['time_to_expiration']<=1)\
                                        & (options_data_calls['time_to_expiration']>=.5)]

    # Next, we set up a regression algorithm to identify the optimal values of the parameters in the least difference square sense.
    def objective(params, option_data, r):                  
        kappa, theta, xi, rho, v0 = params
        
        error = 0    
        for _, row in option_data.iterrows():
            model_price = heston_call(             # Here, we use a semi-closed formula to calculte the Heston price of a European call option (see Functions.py for the formula).      
                S0=row['spot_price'],
                K=row['strike'],
                v0=v0,
                r=r,
                t=row['time_to_expiration'],
                kappa=kappa,
                theta=theta,
                xi=xi,
                rho=rho
            )
            market_price = row['lastPrice']
            error += (model_price - market_price) ** 2    # The loss function here is ||model_price - market_price||_2^2, and we minimize it.
            
        return error / len(option_data)

    # Say, if we want to switch to a different model and calibrate its parameters, then we only need to change the parameters to that of the different model,
    # and then change the model_price function above.

    initial_guess = [1.0, 0.04, 0.3, -0.5, 0.04]  # These are the initial values for the optimization algorithm for finding: kappa, theta, xi, rho, v0.
    bounds = [(1e-4, 10), (1e-4, 1), (1e-4, 2), (-0.99, 0.99), (1e-4, 1)]  # These are the bounds for the above parameters.
    
    
    r = 0.039    
    
    result = minimize(
        objective,          # We are minimizing the value of the (averaged) error function above.
        initial_guess,      # Initial values for parameters.
        args=(options_data_calls, r),    # Our call option prices data set, as well as the discount rate.
        bounds=bounds,
        method='L-BFGS-B',        # The method we choose for the optimization algorithm.
        options={
            'disp': True,   
            'maxiter': 20,       
            'ftol': 1e-4,         
            'gtol': 1e-4          
        }
    )
    
    calibrated_params = result.x

    return calibrated_params


# Below we set up the functions that calculates the option price, option Delta and option Vega. Our calculation shall be vectorized,
# meaning that the input of stock spot price and stock volatility will be the arrays of all the simulated values all at once. This 
# prevents the frequent call for these calculating functions and significantly reduces the running duration of our codes.

def heston_call_price_vec(S0, K, v0, r, t, kappa, theta, xi, rho, phi_max=200, n_phi=2000):
    """
    Vectorized Heston call option price using trapezoidal integration.
    Supports scalar or array input for S0.
    """
    i = 1j
    phi_grid = np.linspace(1e-5, phi_max, n_phi)   # integration grid
    
    # Ensure S0 is array
    S0 = np.atleast_1d(S0)
    v0 = np.atleast_1d(v0)
    log_moneyness = np.log(S0 / K)  # shape (len(S0), n_phi)
    
    def integrand(phi, Pnum):
        u = 0.5 if Pnum == 1 else -0.5
        b = kappa - rho * xi if Pnum == 1 else kappa
        a = kappa * theta

        d = np.sqrt((rho * xi * phi * i - b)**2 - xi**2 * (2 * u * phi * i - phi**2))
        g = (b - rho * xi * phi * i + d) / (b - rho * xi * phi * i - d)

        exp1 = np.exp(i * phi[:, None] * log_moneyness[None, :])  # (n_phi, n_sims)
        C = r * phi[:, None] * i * t + (a / xi**2) * (
            (b - rho * xi * phi[:, None] * i + d[:, None]) * t
            - 2 * np.log((1 - g[:, None] * np.exp(d[:, None] * t)) / (1 - g[:, None]))
        )
        D = (b - rho * xi * phi[:, None] * i + d[:, None]) / xi**2 * (
            (1 - np.exp(d[:, None] * t)) / (1 - g[:, None] * np.exp(d[:, None] * t))
        )

        f = exp1 * np.exp(C + D * v0)
        return np.real(f / (phi[:, None] * i))  # (n_phi, n_sims)

    # Compute integrals
    I1 = np.trapz(np.real(integrand(phi_grid, 1)), phi_grid, axis=0)
    I2 = np.trapz(np.real(integrand(phi_grid, 2)), phi_grid, axis=0)

    P1 = 0.5 + (1/np.pi)*I1 
    P2 = 0.5 + (1/np.pi)*I2

    # Final call
    call = S0 * P1 - K * np.exp(-r*t) * P2             # This is the semi-closed form of European Heston call price.
    return call


def heston_call_delta_vec(S0, K, v0, r, t, kappa, theta, xi, rho, phi_max=200, n_phi=2000):
    """
    Vectorized Heston call option Delta using trapezoidal integration.
    Supports scalar or array input for S0.
    """
    i = 1j
    phi = np.linspace(1e-5, phi_max, n_phi)   # integration grid

    # Make sure S0 is vector, K is scalar
    S0 = np.atleast_1d(S0)  # shape (n_sims,)
    v0 = np.atleast_1d(v0)
    log_moneyness = np.log(S0 / K)  # shape (n_sims,)

    def integrand(phi, Pnum):
        u = 0.5 if Pnum == 1 else -0.5
        b = kappa - rho * xi if Pnum == 1 else kappa
        a = kappa * theta

        d = np.sqrt((rho * xi * phi * i - b)**2 - xi**2 * (2 * u * phi * i - phi**2))
        g = (b - rho * xi * phi * i + d) / (b - rho * xi * phi * i - d)

        exp1 = np.exp(i * phi[:, None] * log_moneyness[None, :])  # (n_phi, n_sims)
        C = r * phi[:, None] * i * t + (a / xi**2) * (
            (b - rho * xi * phi[:, None] * i + d[:, None]) * t
            - 2 * np.log((1 - g[:, None] * np.exp(d[:, None] * t)) / (1 - g[:, None]))
        )
        D = (b - rho * xi * phi[:, None] * i + d[:, None]) / xi**2 * (
            (1 - np.exp(d[:, None] * t)) / (1 - g[:, None] * np.exp(d[:, None] * t))
        )

        f = exp1 * np.exp(C + D * v0)
        return np.real(f / (phi[:, None] * i))  # (n_phi, n_sims)

    def integrand_1(phi, Pnum):
        u = 0.5 if Pnum == 1 else -0.5
        b = kappa - rho * xi if Pnum == 1 else kappa
        a = kappa * theta

        d = np.sqrt((rho * xi * phi * i - b)**2 - xi**2 * (2 * u * phi * i - phi**2))
        g = (b - rho * xi * phi * i + d) / (b - rho * xi * phi * i - d)

        exp1 = np.exp(i * phi[:, None] * log_moneyness[None, :])  # (n_phi, n_sims)
        C = r * phi[:, None] * i * t + (a / xi**2) * (
            (b - rho * xi * phi[:, None] * i + d[:, None]) * t
            - 2 * np.log((1 - g[:, None] * np.exp(d[:, None] * t)) / (1 - g[:, None]))
        )
        D = (b - rho * xi * phi[:, None] * i + d[:, None]) / xi**2 * (
            (1 - np.exp(d[:, None] * t)) / (1 - g[:, None] * np.exp(d[:, None] * t))
        )

        f = exp1 * np.exp(C + D * v0)
        return np.real(f)  # (n_phi, n_sims)

    # Integrals along phi-axis
    I1 = np.trapz(integrand(phi, 1), phi, axis=0)  # (n_sims,)
    I2 = np.trapz(integrand(phi, 2), phi, axis=0)  # (n_sims,)

    J1 = np.trapz(integrand_1(phi, 1), phi, axis=0)  # (n_sims,)
    J2 = np.trapz(integrand_1(phi, 2), phi, axis=0)  # (n_sims,)

    P1 = 0.5 + (1 / np.pi) * I1

    delta = P1 + (1 / np.pi) * (J1 - J2)  # shape (n_sims,)          # This is the semi-closed form of European Heston call price.

    return delta if delta.shape[0] > 1 else delta.item()


def heston_call_vega_vec(S0, K, v0, r, t, kappa, theta, xi, rho, phi_max=200, n_phi=2000):
    """
    Vectorized Heston call option Vega using trapezoidal integration.
    Supports scalar or array input for S0.
    """
    i = 1j
    phi = np.linspace(1e-5, phi_max, n_phi)   # integration grid

    # Make sure S0 is vector, K is scalar
    S0 = np.atleast_1d(S0)  # shape (n_sims,)
    v0 = np.atleast_1d(v0)
    log_moneyness = np.log(S0 / K)  # shape (n_sims,)

    def integrand(phi, Pnum):
        u = 0.5 if Pnum == 1 else -0.5
        b = kappa - rho * xi if Pnum == 1 else kappa
        a = kappa * theta

        d = np.sqrt((rho * xi * phi * i - b)**2 - xi**2 * (2 * u * phi * i - phi**2))
        g = (b - rho * xi * phi * i + d) / (b - rho * xi * phi * i - d)

        exp1 = np.exp(i * phi[:, None] * log_moneyness[None, :])  # (n_phi, n_sims)
        C = r * phi[:, None] * i * t + (a / xi**2) * (
            (b - rho * xi * phi[:, None] * i + d[:, None]) * t
            - 2 * np.log((1 - g[:, None] * np.exp(d[:, None] * t)) / (1 - g[:, None]))
        )
        D = (b - rho * xi * phi[:, None] * i + d[:, None]) / xi**2 * (
            (1 - np.exp(d[:, None] * t)) / (1 - g[:, None] * np.exp(d[:, None] * t))
        )

        f = exp1 * np.exp(C + D * v0)
        return np.real(D*f / (phi[:, None] * i))  # (n_phi, n_sims)

    # Integrals along phi-axis
    I1 = np.trapz(integrand(phi, 1), phi, axis=0)  # (n_sims,)
    I2 = np.trapz(integrand(phi, 2), phi, axis=0)  # (n_sims,)

    vega = S0 * I1 - K * np.exp(-r*t) * I2  # (nS,)        # This is the semi-closed form of European Heston call price.
    return vega
    



def Heston_path_vectorized(S0, T, r, kappa, theta, xi, rho, v0, dt, n_sims):
    """
    Vectorized Heston stock price.
    The output is a matrix consisting of the stock price at the prescribed cutting points in the time interval [0,T] of all the simulations
    (that is, of shape (n_sims, n_steps = int(T/dt))).
    """
    n_steps = int(T/dt)
    
    dW1 = np.random.normal(0,1,size=(n_sims, n_steps)) * np.sqrt(dt)
    dW2 = rho*dW1 + np.sqrt(1 - rho**2) * np.random.normal(0,1,size=(n_sims, n_steps)) * np.sqrt(dt)
    
    S = np.zeros((n_sims, n_steps+1))
    v = np.zeros((n_sims, n_steps+1))
    S[:,0] = S0
    v[:,0] = v0
    
    for j in range(n_steps):
        vt = np.maximum(v[:,j], 0)
        vt_sqrt = np.sqrt(vt)
        v[:, j+1] = np.maximum(vt + kappa*(theta - vt)*dt + xi*vt_sqrt*dW2[:, j], 0)
        S[:, j+1] = S[:, j] * np.exp((r - 0.5*vt)*dt + vt_sqrt*dW1[:, j])             # These steps follows from the discretenization of the 
                                                                                      #defining equations of Heston's model.
    return S, v


# -----------------------------
# Delta-hedge PnL
# -----------------------------
def D_hedge_profit_vectorized(S_path, K, T, r, kappa, theta, xi, rho, v_path, dt, n_hedges):
    """
    For Delta hedging, we only need to hold a varying number of the underlying stock other than the option to achieve Delta neutrality.
    """
    
    n_sims = S_path.shape[0]
    scale = int(S_path.shape[1] / n_hedges)
    
    t_b = np.arange(n_hedges-1) * scale        # We set up n_hedges hedging time points with equal gaps in the interval [0,T].
    t_f = np.arange(1, n_hedges) * scale
    
    S_b = S_path[:, t_b]
    S_f = S_path[:, t_f]
    v_b = v_path[:, t_b]
    
    discount_b = np.exp(-r * t_b * dt)
    discount_f = np.exp(-r * t_f * dt)
    
    hedge_step_profit = np.zeros((n_sims, n_hedges-1))
    
    for i in range(n_hedges-1):
        T_remaining = T - t_b[i] * dt
        
        delta_i = heston_call_delta_vec(S_b[:,i], K, v_b[:,i], r, T_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        hedge_step_profit[:, i] = (discount_f[i]*S_f[:, i] - discount_b[i]*S_b[:, i]) * (-delta_i)    # This gives the hedging revenue between the i-th and (i+1)-th hedging points, with discount.
        
    return hedge_step_profit      

# -----------------------------
# Vega-hedge PnL
# -----------------------------

def V_hedge_profit_vectorized(S_path, K, K_1, T, T_1, r, kappa, theta, xi, rho, v_path, dt, n_hedges):
    """
    For Vega hedging, we need to hold a varying number of another European option on the same underlying stock but have later maturities
    other than the option to achieve Vega neutrality.
    """
    n_sims = S_path.shape[0]
    scale = int(S_path.shape[1] / n_hedges)
    
    t_b = np.arange(n_hedges-1) * scale
    t_f = np.arange(1, n_hedges) * scale
    
    S_b = S_path[:, t_b]
    S_f = S_path[:, t_f]
    v_b = v_path[:, t_b]
    v_f = v_path[:, t_f]
    
    discount_b = np.exp(-r * t_b * dt)
    discount_f = np.exp(-r * t_f * dt)
    
    hedge_step_profit = np.zeros((n_sims, n_hedges-1))
    
    for i in range(n_hedges-1):
        T_remaining = T - t_b[i] * dt
        T_1_remaining = T_1 - t_b[i] * dt
        
        C_vega_i = heston_call_vega_vec(S_b[:,i], K, v_b[:,i], r, T_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        C1_vega_i = heston_call_vega_vec(S_b[:,i], K_1, v_b[:,i], r, T_1_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        C1_vega_i = np.where(C1_vega_i != 0, C1_vega_i, 0.01)
        
        price_f_i = heston_call_price_vec(S_f[:,i], K_1, v_f[:,i], r, T_1_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)
        price_b_i = heston_call_price_vec(S_b[:,i], K_1, v_b[:,i], r, T_1_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)
        
        hedge_step_profit[:, i] = (discount_f[i]*price_f_i - discount_b[i]*price_b_i) * (-C_vega_i/C1_vega_i)   # This gives the hedging revenue between the i-th and (i+1)-th hedging points, with discount.

    return hedge_step_profit


# -----------------------------
# Delta-Vega-hedge PnL
# -----------------------------
def DV_hedge_profit_vectorized(S_path, K, K_1, K_2, T, T_1, T_2, r, kappa, theta, xi, rho, v_path, dt, n_hedges):
    """
    For Delta-Vega hedging, we need to hold varying numbers of two other European options on the same underlying stock but have later maturities
    other than the option to achieve Delta and Vega neutrality.
    """
    n_sims = S_path.shape[0]
    scale = int(S_path.shape[1] / n_hedges)
    
    t_b = np.arange(n_hedges-1) * scale
    t_f = np.arange(1, n_hedges) * scale
    
    S_b = S_path[:, t_b]
    S_f = S_path[:, t_f]
    v_b = v_path[:, t_b]
    v_f = v_path[:, t_f]
    
    discount_b = np.exp(-r * t_b * dt)
    discount_f = np.exp(-r * t_f * dt)
    
    hedge_step_profit = np.zeros((n_sims, n_hedges-1))
    
    for i in range(n_hedges-1):
        T_remaining = T - t_b[i] * dt
        T_1_remaining = T_1 - t_b[i] * dt
        T_2_remaining = T_2 - t_b[i] * dt
        
        vega_i = heston_call_vega_vec(S_b[:,i], K, v_b[:,i], r, T_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        vega1_i = heston_call_vega_vec(S_b[:,i], K_1, v_b[:,i], r, T_1_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        vega2_i = heston_call_vega_vec(S_b[:,i], K_2, v_b[:,i], r, T_2_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        vega2_i = np.where(vega2_i != 0, vega2_i, 0.01)

        delta_i = heston_call_delta_vec(S_b[:,i], K, v_b[:,i], r, T_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        delta1_i = heston_call_delta_vec(S_b[:,i], K_1, v_b[:,i], r, T_1_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        delta2_i = heston_call_delta_vec(S_b[:,i], K_2, v_b[:,i], r, T_2_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        
        price1_f_i = heston_call_price_vec(S_f[:,i], K_1, v_f[:,i], r, T_1_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)
        price1_b_i = heston_call_price_vec(S_b[:,i], K_1, v_b[:,i], r, T_1_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)

        price2_f_i = heston_call_price_vec(S_f[:,i], K_2, v_f[:,i], r, T_2_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)
        price2_b_i = heston_call_price_vec(S_b[:,i], K_2, v_b[:,i], r, T_2_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)

        deno_i = delta1_i - delta2_i*vega1_i/vega2_i
        deno_i = np.where(deno_i != 0, deno_i, 0.01)

        alpha_i = (vega_i*delta2_i/vega2_i - delta_i)/deno_i
        beta_i = -(vega_i + alpha_i*vega1_i)/vega2_i
        
        hedge_step_profit[:, i] = (discount_f[i]*price1_f_i - discount_b[i]*price1_b_i) * alpha_i \
        + (discount_f[i]*price2_f_i - discount_b[i]*price2_b_i) * beta_i    # This gives the hedging revenue between the i-th and (i+1)-th hedging points, with discount.
    
    return hedge_step_profit

# -----------------------------
# Delta Hedged Portfolio profits
# -----------------------------
def delta_hedged_portfolio_profits_vectorized(S0, K, T, r, kappa, theta, xi, rho, v0, dt, n_sims, n_hedges):
    """
    Calculate the total profit of a Delta hedging portfolio.
    """
    S_path, v_path = Heston_path_vectorized(S0, T, r, kappa, theta, xi, rho, v0, dt, n_sims)
    hedge_step_profit = D_hedge_profit_vectorized(S_path, K, T, r, kappa, theta, xi, rho, v_path, dt, n_hedges)  
    hedge_total_profit = np.sum(hedge_step_profit, axis=1)    # Adding up all the hedging profits.
    payoff = np.maximum(S_path[:, -1] - K, 0)                     #option payoff.
    portfolio_profit = hedge_total_profit + payoff * np.exp(-r*T)    # Total profits = hedging profits + option payoff.
    return portfolio_profit

# -----------------------------
# Vega Hedged Portfolio profits
# -----------------------------
def vega_hedged_portfolio_profits_vectorized(S0, K, K_1, T, T_1, r, kappa, theta, xi, rho, v0, dt, n_sims, n_hedges):
    """
    Calculate the total profit of a Vega hedging portfolio.
    """
    S_path, v_path = Heston_path_vectorized(S0, T, r, kappa, theta, xi, rho, v0, dt, n_sims)
    hedge_step_profit = V_hedge_profit_vectorized(S_path, K, K_1, T, T_1, r, kappa, theta, xi, rho, v_path, dt, n_hedges)
    hedge_total_profit = np.sum(hedge_step_profit, axis=1)
    payoff = np.maximum(S_path[:, -1] - K, 0)
    portfolio_profit = hedge_total_profit + payoff * np.exp(-r*T)
    return portfolio_profit

# -----------------------------
# Delta-Vega Hedged Portfolio profits
# -----------------------------
def delta_vega_hedged_portfolio_profits_vectorized(S0, K, K_1, K_2, T, T_1, T_2, r, kappa, theta, xi, rho, v0, dt, n_sims, n_hedges):
    """
    Calculate the total profit of a Delta-Vega hedging portfolio.
    """
    S_path, v_path = Heston_path_vectorized(S0, T, r, kappa, theta, xi, rho, v0, dt, n_sims)
    hedge_step_profit = DV_hedge_profit_vectorized(S_path, K, K_1, K_2, T, T_1, T_2, r, kappa, theta, xi, rho, v_path, dt, n_hedges)
    hedge_total_profit = np.sum(hedge_step_profit, axis=1)
    payoff = np.maximum(S_path[:, -1] - K, 0)
    portfolio_profit = hedge_total_profit + payoff * np.exp(-r*T)
    return portfolio_profit


